In [ ]:
#!/usr/bin/env python3
#coding=utf-8
import subprocess
import wave
import time
import os
from aip import AipSpeech
import threading


In [ ]:
# ==================== 系统配置和初始化 ====================
print("[初始化] 开始语音对话系统初始化...")

# 百度语音识别 API 的配置信息
APP_ID = '90192389'
API_KEY = 'Fg9ZtQk3ymOq5ztw2QG0Ffiu'
SECRET_KEY = '8WXF8pgIxGzdvMikguWzx6AUx8mRtmaB'

# 初始化 AipSpeech 客户端
try:
    client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)
    print("[初始化] ✓ 百度语音API客户端初始化成功")
except Exception as e:
    print(f"[初始化] ✗ 百度语音API客户端初始化失败: {e}")
    client = None

print("[初始化] 语音对话系统初始化完成")
print("=" * 50)


In [ ]:
# ==================== 音频设备详细检测 ====================
def detailed_audio_device_check():
    print("[音频检测] 开始详细音频设备检测...")
    
    # 检测录音设备
    print("\n[音频检测] === 录音设备检测 ===")
    try:
        result = subprocess.run(['arecord', '-l'], capture_output=True, text=True)
        if result.returncode == 0:
            print("[音频检测] ✓ 系统录音设备列表:")
            card_found = False
            for line in result.stdout.split('\n'):
                if line.strip():
                    print(f"    {line}")
                    if 'card' in line.lower():
                        card_found = True
            
            if not card_found:
                print("[音频检测] ⚠ 未找到可用的录音设备")
        else:
            print(f"[音频检测] ✗ 无法获取录音设备列表: {result.stderr}")
    except FileNotFoundError:
        print("[音频检测] ✗ arecord命令未找到，请确认ALSA工具已安装")
    except Exception as e:
        print(f"[音频检测] ✗ 录音设备检测异常: {e}")
    
    # 检测播放设备
    print("\n[音频检测] === 播放设备检测 ===")
    try:
        result = subprocess.run(['aplay', '-l'], capture_output=True, text=True)
        if result.returncode == 0:
            print("[音频检测] ✓ 系统播放设备列表:")
            card_found = False
            for line in result.stdout.split('\n'):
                if line.strip():
                    print(f"    {line}")
                    if 'card' in line.lower():
                        card_found = True
            
            if not card_found:
                print("[音频检测] ⚠ 未找到可用的播放设备")
        else:
            print(f"[音频检测] ✗ 无法获取播放设备列表: {result.stderr}")
    except FileNotFoundError:
        print("[音频检测] ✗ aplay命令未找到，请确认ALSA工具已安装")
    except Exception as e:
        print(f"[音频检测] ✗ 播放设备检测异常: {e}")
    
    # 测试默认录音设备
    print("\n[音频检测] === 录音设备测试 ===")
    test_devices = ['default', 'plughw:0,0', 'plughw:1,0', 'plughw:2,0']
    working_device = None
    
    for device in test_devices:
        try:
            print(f"[音频检测] 测试录音设备: {device}")
            test_command = ['arecord', '-D', device, '-f', 'S16_LE', '-r', '16000', '-c', '1', '-d', '1', '/tmp/test_audio.wav']
            result = subprocess.run(test_command, capture_output=True, timeout=3)
            
            if result.returncode == 0:
                print(f"[音频检测] ✓ 设备 {device} 可用")
                working_device = device
                # 清理测试文件
                try:
                    os.remove('/tmp/test_audio.wav')
                except:
                    pass
                break
            else:
                print(f"[音频检测] ✗ 设备 {device} 不可用: {result.stderr.decode('utf-8', errors='ignore')[:100]}")
        except subprocess.TimeoutExpired:
            print(f"[音频检测] ✗ 设备 {device} 测试超时")
        except Exception as e:
            print(f"[音频检测] ✗ 设备 {device} 测试异常: {e}")
    
    if working_device:
        print(f"\n[音频检测] ✓ 推荐使用录音设备: {working_device}")
        return working_device
    else:
        print("\n[音频检测] ✗ 未找到可用的录音设备")
        return 'default'

# 执行设备检测
recommended_device = detailed_audio_device_check()
print(f"\n[音频检测] 系统推荐录音设备: {recommended_device}")
print("=" * 50)


In [ ]:
# ==================== 语音录制与识别函数 ====================
def record_and_recognize_speech(record_duration=4, device=None):
    """
    录制音频并转换为文字
    
    Args:
        record_duration: 录音时长(秒)
        device: 录音设备，None时使用推荐设备
    
    Returns:
        识别的文字内容，失败返回None
    """
    if device is None:
        device = recommended_device
    
    print(f"\n[语音识别] 开始录音，时长: {record_duration}秒，设备: {device}")
    print("[语音识别] 请开始说话...")
    
    # 定义录音命令和参数
    audio_file = 'voice_record.wav'
    command = [
        'arecord',
        '-D', device,
        '-f', 'S16_LE',
        '-r', '16000',
        '-c', '1',
        '-d', str(record_duration),
        audio_file
    ]
    
    # 百度API参数
    options = {
        'dev_pid': 1536,  # 普通话输入法模型
        'lm_id': 0        # 默认语言模型
    }
    
    wf = None
    
    try:
        # 执行录音
        print(f"[语音识别] 执行命令: {' '.join(command)}")
        start_time = time.time()
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        record_time = time.time() - start_time
        
        print(f"[语音识别] 录音耗时: {record_time:.2f}秒")
        
        # 检查录音结果
        if process.returncode != 0:
            error_msg = stderr.decode('utf-8', errors='ignore')
            print(f"[语音识别] ✗ 录音失败 (返回码: {process.returncode})")
            print(f"[语音识别] 错误信息: {error_msg[:200]}")
            return None
        
        print("[语音识别] ✓ 录音完成")
        
        # 检查音频文件
        if not os.path.exists(audio_file):
            print(f"[语音识别] ✗ 音频文件不存在: {audio_file}")
            return None
        
        file_size = os.path.getsize(audio_file)
        print(f"[语音识别] 音频文件大小: {file_size} bytes")
        
        if file_size < 1000:
            print("[语音识别] ✗ 音频文件太小，可能录音失败")
            return None
        
        # 分析音频文件
        try:
            wf = wave.open(audio_file, 'rb')
            
            # 获取音频参数
            channels = wf.getnchannels()
            sample_width = wf.getsampwidth()
            framerate = wf.getframerate()
            frames = wf.getnframes()
            duration = frames / framerate
            
            print(f"[语音识别] 音频参数分析:")
            print(f"    通道数: {channels}")
            print(f"    位深: {sample_width * 8} bit")
            print(f"    采样率: {framerate} Hz")
            print(f"    帧数: {frames}")
            print(f"    实际时长: {duration:.2f} 秒")
            
            # 检查音频质量
            if duration < 0.5:
                print("[语音识别] ⚠ 音频时长过短，可能影响识别效果")
            elif duration > record_duration + 1:
                print("[语音识别] ⚠ 音频时长异常")
            
            # 读取音频数据
            FORMAT = 'wav' if sample_width == 2 else 'pcm'
            audio_data = wf.readframes(frames)
            print(f"[语音识别] 音频数据大小: {len(audio_data)} bytes，格式: {FORMAT}")
            
        except Exception as e:
            print(f"[语音识别] ✗ 音频文件分析失败: {e}")
            return None
        
        # 调用百度语音识别API
        print("[语音识别] 开始调用百度语音识别API...")
        api_start_time = time.time()
        
        if not client:
            print("[语音识别] ✗ 百度API客户端未初始化")
            return None
        
        try:
            result = client.asr(audio_data, FORMAT, framerate, options)
            api_time = time.time() - api_start_time
            print(f"[语音识别] API调用耗时: {api_time:.2f}秒")
            
        except Exception as e:
            print(f"[语音识别] ✗ API调用异常: {e}")
            return None
        
        print(f"[语音识别] API返回结果: {result}")
        
        # 解析识别结果
        if isinstance(result, dict):
            if 'err_no' in result and result['err_no'] == 0:
                if 'result' in result and len(result['result']) > 0:
                    recognized_text = result['result'][0]
                    print(f"[语音识别] ✓ 识别成功: '{recognized_text}'")
                    print(f"[语音识别] 置信度: {result.get('corpus_no', 'N/A')}")
                    return recognized_text
                else:
                    print("[语音识别] ✗ API返回为空结果")
                    return None
            else:
                error_code = result.get('err_no', 'unknown')
                error_msg = result.get('err_msg', '未知错误')
                print(f"[语音识别] ✗ API识别失败 (错误码: {error_code}): {error_msg}")
                return None
        else:
            print(f"[语音识别] ✗ API返回格式异常: {type(result)}")
            return None
        
    except Exception as e:
        print(f"[语音识别] ✗ 程序异常: {str(e)}")
        return None
    
    finally:
        # 清理资源
        if wf:
            try:
                wf.close()
            except:
                pass
        
        # 清理临时文件
        try:
            if os.path.exists(audio_file):
                os.remove(audio_file)
                print(f"[语音识别] 清理临时文件: {audio_file}")
        except Exception as e:
            print(f"[语音识别] 清理文件失败: {e}")


In [ ]:
# ==================== 语音合成与播放函数 ====================
def synthesize_and_play_speech(text, voice_speed=5, voice_pitch=5, voice_volume=15):
    """
    将文字转换为语音并播放
    
    Args:
        text: 要播放的文字
        voice_speed: 语速 (0-15)
        voice_pitch: 音调 (0-15) 
        voice_volume: 音量 (0-15)
    
    Returns:
        True表示成功，False表示失败
    """
    if not text:
        print("[语音播报] ✗ 文字内容为空")
        return False
    
    if not client:
        print("[语音播报] ✗ 百度API客户端未初始化")
        return False
    
    print(f"\n[语音播报] 开始合成语音: '{text}'")
    print(f"[语音播报] 参数 - 语速:{voice_speed}, 音调:{voice_pitch}, 音量:{voice_volume}")
    
    output_file = 'voice_output.wav'
    
    try:
        # 调用百度语音合成API
        synthesis_options = {
            'aue': 6,              # wav(pcm-16k)格式
            'spd': voice_speed,    # 语速
            'pit': voice_pitch,    # 音调
            'vol': voice_volume,   # 音量
            'per': 0               # 发音人选择，0为度小美
        }
        
        print("[语音播报] 调用百度语音合成API...")
        synthesis_start_time = time.time()
        
        try:
            res = client.synthesis(text, 'zh', 1, synthesis_options)
            synthesis_time = time.time() - synthesis_start_time
            print(f"[语音播报] 语音合成耗时: {synthesis_time:.2f}秒")
            
        except Exception as e:
            print(f"[语音播报] ✗ API调用异常: {e}")
            return False
        
        # 检查合成结果
        if isinstance(res, dict):
            error_code = res.get('err_no', 'unknown')
            error_msg = res.get('err_msg', '未知错误')
            print(f"[语音播报] ✗ 语音合成失败 (错误码: {error_code}): {error_msg}")
            return False
        
        if not isinstance(res, bytes):
            print(f"[语音播报] ✗ 返回数据格式错误: {type(res)}")
            return False
        
        print(f"[语音播报] ✓ 语音合成成功，数据大小: {len(res)} bytes")
        
        # 保存音频文件
        try:
            with open(output_file, 'wb') as f:
                f.write(res)
            print(f"[语音播报] ✓ 音频文件保存成功: {output_file}")
            
            # 验证文件大小
            file_size = os.path.getsize(output_file)
            print(f"[语音播报] 文件大小验证: {file_size} bytes")
            
            if file_size < 1000:
                print("[语音播报] ⚠ 音频文件太小，可能有问题")
            
        except Exception as e:
            print(f"[语音播报] ✗ 保存音频文件失败: {e}")
            return False
        
        # 播放音频
        play_command = ['aplay', output_file]
        print(f"[语音播报] 执行播放命令: {' '.join(play_command)}")
        
        try:
            play_start_time = time.time()
            process = subprocess.Popen(play_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout, stderr = process.communicate()
            play_time = time.time() - play_start_time
            
            print(f"[语音播报] 播放耗时: {play_time:.2f}秒")
            
            if process.returncode == 0:
                print("[语音播报] ✓ 播放完成")
                return True
            else:
                error_msg = stderr.decode('utf-8', errors='ignore')
                print(f"[语音播报] ✗ 播放失败 (返回码: {process.returncode})")
                print(f"[语音播报] 错误信息: {error_msg[:200]}")
                return False
                
        except Exception as e:
            print(f"[语音播报] ✗ 播放异常: {e}")
            return False
        
    except Exception as e:
        print(f"[语音播报] ✗ 程序异常: {str(e)}")
        return False
    
    finally:
        # 清理音频文件
        try:
            if os.path.exists(output_file):
                os.remove(output_file)
                print(f"[语音播报] 清理临时文件: {output_file}")
        except Exception as e:
            print(f"[语音播报] 清理文件失败: {e}")


In [ ]:
# ==================== 智能对话回复生成 ====================
def generate_chat_response(user_input):
    """
    根据用户输入生成智能回复
    
    Args:
        user_input: 用户的语音输入文字
    
    Returns:
        回复的文字内容
    """
    if not user_input:
        return "我没有听清楚，请再说一遍"
    
    user_input = user_input.strip().lower()
    print(f"[对话生成] 处理用户输入: '{user_input}'")
    
    # 问候类
    if any(word in user_input for word in ['你好', 'hello', '嗨', '您好']):
        responses = [
            "你好！很高兴和您对话",
            "您好！我是您的语音助手",
            "嗨！有什么可以帮助您的吗？"
        ]
        import random
        return random.choice(responses)
    
    # 自我介绍类
    elif any(word in user_input for word in ['你是谁', '名字', '叫什么', '自我介绍']):
        return "我是一个语音对话助手，可以和您进行简单的对话交流"
    
    # 年龄询问
    elif any(word in user_input for word in ['多少岁', '年龄', '几岁']):
        return "我是一个人工智能程序，没有年龄概念，但我每天都在学习新知识"
    
    # 能力询问
    elif any(word in user_input for word in ['功能', '能力', '会什么', '能做什么']):
        return "我可以听懂您的话，并和您进行对话交流。我还在不断学习更多技能"
    
    # 天气相关
    elif any(word in user_input for word in ['天气', '下雨', '晴天', '温度']):
        return "抱歉，我暂时无法获取天气信息，建议您查看天气应用"
    
    # 时间相关
    elif any(word in user_input for word in ['时间', '几点', '现在']):
        import datetime
        now = datetime.datetime.now()
        return f"现在是{now.strftime('%H点%M分')}"
    
    # 日期相关
    elif any(word in user_input for word in ['日期', '今天', '几号']):
        import datetime
        today = datetime.datetime.now()
        return f"今天是{today.strftime('%Y年%m月%d日')}"
    
    # 喜好询问
    elif any(word in user_input for word in ['喜欢', '爱好', '兴趣']):
        return "我喜欢和人类对话，学习新知识。我对科技和语言特别感兴趣"
    
    # 学习相关
    elif any(word in user_input for word in ['学习', '读书', '知识']):
        return "我认为学习很重要，通过不断学习可以让我们变得更优秀"
    
    # 感谢
    elif any(word in user_input for word in ['谢谢', '感谢', 'thanks']):
        return "不用客气！很高兴能帮助到您"
    
    # 再见
    elif any(word in user_input for word in ['再见', '拜拜', '退出', '结束', 'bye']):
        return "再见！期待下次和您对话"
    
    # 夸奖
    elif any(word in user_input for word in ['聪明', '厉害', '棒', '不错']):
        return "谢谢您的夸奖！我会继续努力的"
    
    # 询问心情
    elif any(word in user_input for word in ['心情', '开心', '高兴', '难过']):
        return "我没有情感，但我很高兴能和您交流。希望您每天都心情愉快"
    
    # 吃饭相关
    elif any(word in user_input for word in ['吃饭', '饿了', '食物', '美食']):
        return "我不需要吃饭，但我听说美食能让人心情愉快"
    
    # 睡觉相关
    elif any(word in user_input for word in ['睡觉', '累了', '困了', '休息']):
        return "适当的休息很重要，祝您睡个好觉"
    
    # 音乐相关
    elif any(word in user_input for word in ['音乐', '歌曲', '唱歌']):
        return "我无法播放音乐，但我知道音乐能够陶冶情操，让人放松"
    
    # 游戏相关
    elif any(word in user_input for word in ['游戏', '玩', '娱乐']):
        return "适度游戏有益身心，但记得劳逸结合哦"
    
    # 工作相关
    elif any(word in user_input for word in ['工作', '上班', '职业']):
        return "工作很重要，但也要注意身体健康。希望您工作顺利"
    
    # 家庭相关
    elif any(word in user_input for word in ['家人', '父母', '家庭']):
        return "家人很重要，多陪伴家人，珍惜在一起的时光"
    
    # 朋友相关
    elif any(word in user_input for word in ['朋友', '同事', '关系']):
        return "好朋友是人生的财富，要珍惜身边的朋友"
    
    # 健康相关
    elif any(word in user_input for word in ['健康', '身体', '运动']):
        return "健康最重要，记得多运动，保持良好的生活习惯"
    
    # 旅行相关
    elif any(word in user_input for word in ['旅行', '旅游', '出去玩']):
        return "旅行能开阔视野，增长见识。祝您旅途愉快"
    
    # 默认回复
    else:
        default_responses = [
            "这很有趣，请继续说下去",
            "我还在学习理解这个话题，能告诉我更多吗？",
            "听起来很不错，您能详细说说吗？",
            "这是个有趣的观点",
            "我明白了，还有其他想说的吗？",
            "谢谢您告诉我这些",
            "这让我学到了新知识"
        ]
        import random
        return random.choice(default_responses)


In [ ]:
# ==================== 系统状态检查 ====================
def system_health_check():
    """
    系统健康状态检查
    """
    print("\n" + "=" * 60)
    print("[状态检查] 语音对话系统健康检查")
    print("=" * 60)
    
    # 检查百度API状态
    print("\n[状态检查] === 百度语音API检查 ===")
    if client:
        try:
            # 测试语音合成
            test_text = "系统测试"
            result = client.synthesis(test_text, 'zh', 1, {'aue': 6})
            if isinstance(result, bytes) and len(result) > 1000:
                print("[状态检查] ✓ 百度语音合成API正常")
                print(f"    测试合成数据大小: {len(result)} bytes")
            else:
                print(f"[状态检查] ✗ 百度语音合成API异常: {result}")
        except Exception as e:
            print(f"[状态检查] ✗ 百度API测试失败: {e}")
    else:
        print("[状态检查] ✗ 百度API客户端未初始化")
    
    # 检查音频播放
    print("\n[状态检查] === 音频播放测试 ===")
    try:
        test_result = synthesize_and_play_speech("测试音频播放", voice_speed=8)
        if test_result:
            print("[状态检查] ✓ 音频播放功能正常")
        else:
            print("[状态检查] ✗ 音频播放功能异常")
    except Exception as e:
        print(f"[状态检查] ✗ 音频播放测试异常: {e}")
    
    # 检查录音功能
    print("\n[状态检查] === 录音功能测试 ===")
    print("[状态检查] 请说话进行录音测试（2秒）...")
    try:
        test_text = record_and_recognize_speech(record_duration=2)
        if test_text:
            print(f"[状态检查] ✓ 录音识别功能正常，识别结果: '{test_text}'")
        else:
            print("[状态检查] ⚠ 录音识别未获得结果，可能是环境太安静或设备问题")
    except Exception as e:
        print(f"[状态检查] ✗ 录音功能测试异常: {e}")
    
    # 检查对话生成
    print("\n[状态检查] === 对话生成测试 ===")
    try:
        test_response = generate_chat_response("你好")
        print(f"[状态检查] ✓ 对话生成功能正常，测试回复: '{test_response}'")
    except Exception as e:
        print(f"[状态检查] ✗ 对话生成测试异常: {e}")
    
    print("\n" + "=" * 60)
    print("[状态检查] 系统健康检查完成")
    print("=" * 60)

# 执行系统检查
system_health_check()


In [ ]:
# ==================== 单次对话测试 ====================
def single_voice_chat():
    """
    单次语音对话测试
    """
    print("\n" + "=" * 50)
    print("[单次测试] 语音对话测试")
    print("请在3秒内说话...")
    print("=" * 50)
    
    try:
        # 录制并识别语音
        user_speech = record_and_recognize_speech(record_duration=3)
        
        if user_speech:
            print(f"\n[单次测试] 用户说话: '{user_speech}'")
            
            # 生成回复
            response = generate_chat_response(user_speech)
            print(f"[单次测试] 系统回复: '{response}'")
            
            # 播放回复
            success = synthesize_and_play_speech(response)
            
            if success:
                print("[单次测试] ✓ 对话测试完成")
            else:
                print("[单次测试] ⚠ 语音播放失败，但对话生成正常")
        else:
            print("[单次测试] ✗ 未识别到语音内容")
            synthesize_and_play_speech("抱歉，我没有听清楚您说的话")
    
    except Exception as e:
        print(f"[单次测试] ✗ 测试异常: {e}")
    
    print("=" * 50)

# 执行单次测试
single_voice_chat()


In [ ]:
# ==================== 连续对话主程序 ====================
def continuous_voice_chat():
    """
    连续语音对话程序
    """
    print("\n" + "=" * 60)
    print("[主程序] 语音对话系统启动")
    print("=" * 60)
    print("功能说明：")
    print("- 支持自然语音对话")
    print("- 智能回复生成")
    print("- 说'再见'、'退出'或'结束'可以结束对话")
    print("=" * 60)
    
    conversation_count = 0
    
    try:
        while True:
            conversation_count += 1
            print(f"\n[对话-{conversation_count}] 请说话（4秒录音）...")
            
            # 录制并识别用户语音
            user_speech = record_and_recognize_speech(record_duration=4)
            
            if user_speech:
                print(f"[对话-{conversation_count}] 用户: {user_speech}")
                
                # 检查是否为退出指令
                if any(word in user_speech.lower() for word in ['再见', '拜拜', '退出', '结束', 'bye']):
                    print("[主程序] 检测到退出指令")
                    final_response = generate_chat_response(user_speech)
                    print(f"[对话-{conversation_count}] 系统: {final_response}")
                    synthesize_and_play_speech(final_response)
                    break
                
                # 生成智能回复
                response = generate_chat_response(user_speech)
                print(f"[对话-{conversation_count}] 系统: {response}")
                
                # 播放回复
                success = synthesize_and_play_speech(response)
                
                if not success:
                    print(f"[对话-{conversation_count}] ⚠ 语音播放失败")
                
            else:
                print(f"[对话-{conversation_count}] 未识别到语音，跳过此轮")
                synthesize_and_play_speech("我没有听清楚，请再说一遍")
            
            # 短暂停顿
            time.sleep(1)
    
    except KeyboardInterrupt:
        print("\n[主程序] 检测到键盘中断 (Ctrl+C)")
        synthesize_and_play_speech("对话被中断，再见")
    
    except Exception as e:
        print(f"\n[主程序] ✗ 程序异常: {e}")
    
    finally:
        print(f"\n[主程序] 对话结束，共进行了 {conversation_count} 轮对话")
        print("[主程序] 感谢使用语音对话系统！")

# 启动连续对话（取消注释下面一行来运行）
# continuous_voice_chat()


In [ ]:
# ==================== 资源清理和系统信息 ====================
def cleanup_resources():
    """
    清理系统资源
    """
    print("\n[清理] 开始清理系统资源...")
    
    # 清理临时文件
    temp_files = ['voice_record.wav', 'voice_output.wav', '/tmp/test_audio.wav']
    for file in temp_files:
        try:
            if os.path.exists(file):
                os.remove(file)
                print(f"[清理] ✓ 删除临时文件: {file}")
        except Exception as e:
            print(f"[清理] ⚠ 删除文件失败 {file}: {e}")
    
    print("[清理] 资源清理完成")

# 注册程序退出时的清理函数
import atexit
atexit.register(cleanup_resources)

# 显示系统就绪信息
print("\n" + "=" * 60)
print("[系统] 语音对话系统已就绪")
print("=" * 60)
print("可用功能:")
print("1. single_voice_chat() - 单次语音对话测试")
print("2. continuous_voice_chat() - 连续语音对话")
print("3. system_health_check() - 系统健康检查")
print("4. detailed_audio_device_check() - 重新检测音频设备")
print("5. record_and_recognize_speech() - 单独测试语音识别")
print("6. synthesize_and_play_speech('文字') - 单独测试语音播放")
print("=" * 60)
print(f"推荐录音设备: {recommended_device}")
print("系统初始化完成，可以开始使用！")
print("=" * 60)
